# Rename files for input4mips

In [1]:
from datetime import datetime
import netCDF4 as nc
import uuid
import os

In [2]:
infolder_list = [
    "../output/aerosols/", 
    "../output/aviation/", 
    "../output/GHGs/"
]
outfolder = "../output/input4mips/"
versions_list = [
    "v4.8.1",
    "v4.8",
    "v4"
]
print_version = "v4-8-1"
source_version= "v4.8.1"
assert len(versions_list) == len(infolder_list)

In [3]:
target_mip = "DAMIP"
old_long_scen_name = "IAMC-MESSAGE-GLOBIOM_SSP2-45-SPA2"
for ind in range(len(infolder_list)):
    infolder = infolder_list[ind]
    version = versions_list[ind]
    files = os.listdir(infolder)
    files = [file for file in files if (file.__contains__(version))]

    for file in files:
        compress = True
        newfilename = file[4:].split(".nc")[0]
        if infolder.__contains__("aerosols"):
            scenario = file.split(".nc_")[1]
            scenario = scenario[:-3]
        elif infolder.__contains__("aviation"):
            scenario = file.split("_" + version)[0].split("_")[-1]
            newfilename = newfilename.replace(scenario, "")
            if scenario.__contains__("4year"):
                scenario = "FourYearBlip"
            elif scenario.__contains__("baseline"):
                scenario = "baseline"
            elif scenario.__contains__("mp06"):
                scenario = "TwoYearBlip"
            else:
                raise ValueError
        elif infolder.__contains__("GHGs"):
            scenario = file.replace("_" + version, "").split(".nc")[1]
        if file.__contains__("baseline") | file.__contains__("Base"):
            print("Skip file {}".format(file))
            continue
        elif scenario == "TwoYearBlip":
            scenario_label = "ssp245-covid"
        else:
            continue
            """
        elif scenario == "FourYearBlip":
            scenario_label = "ssp245-covid4y"
        elif scenario == "ModerateGreen":
            scenario_label = "ssp245-cov-modgreen"
        elif scenario == "FossilFuel":
            scenario_label = "ssp245-cov-fossil"
        elif scenario == "StrongGreen":
            scenario_label = "ssp245-cov-strgreen"
        else:
            print("Cannot find the name for {}".format(scenario))
            raise ValueError
        """
        long_new_label = "ImperialCollege-" + scenario_label + "-" + print_version
        newfilename = newfilename.replace(scenario + "_", "").replace("ScenarioMIP", target_mip).replace(
            "ScenarioMIP", target_mip
        ).replace(
            "-1-1", "").replace("-1-2-1", "").replace("IAMC-", "").replace("UoM-", ""
        ).replace(
            "MESSAGE-GLOBIOM-ssp245", long_new_label
        ).replace("4year_", "").replace("201501-210012", "201501-205012") + ".nc"
        print(scenario)
        print(newfilename)

        ## Copy the previous file details

        db = nc.Dataset(os.path.join(infolder, file))
        trg = nc.Dataset(os.path.join(outfolder, newfilename), mode="w")
        # Create the dimensions of the file
        for name, dim in db.dimensions.items():
            trg.createDimension(
                name, len(dim) if not (dim.isunlimited()) and not (name == "time") else None
            )

        # Copy the global attributes
        trg.setncatts({a: db.getncattr(a) for a in db.ncattrs()})

        for name, var in db.variables.items():
            if compress:
                trg.createVariable(
                    name, var.dtype, var.dimensions, complevel=9, shuffle=True, zlib=True
                )
            else:
                trg.createVariable(name, var.dtype, var.dimensions)

            # Copy the variable attributes
            trg.variables[name].setncatts({a: var.getncattr(a) for a in var.ncattrs()})

            # Copy the variables values, removing some times if needed
            trg.variables[name][:] = db.variables[name][:]


        # Rewrite the meta variables
        trg.comment = db.comment.replace(
            old_long_scen_name, scenario + ", based on " + old_long_scen_name + 
            " modified by the impacts of lockdown for coronavirus"
        )
        trg.contact = "Robin D. Lamboll (r.lamboll@imperial.ac.uk)"
        trg.creation_date = datetime.now().strftime("%Y-%m-%dT%H:%M:%SZ")
        trg.institution = "Imperial College London, South Kensington Campus, London SW7 2AZ, UK"
        trg.institution_id = "ImperialCollege"
        trg.references = "https://doi.org/10.5194/gmd-2020-373"
        trg.source_id = long_new_label
        trg.source_version = source_version
        trg.license = trg.license.replace("ScenarioMIP", target_mip).replace(
            "the IAMC", "ICL (derived from IAMC scenario data)"
        ).replace("UoM", "ICL (derived from UoM scenario data)")
        trg.target_mip = "DAMIP"
        uuidVal = str(uuid.uuid4())
        trg.tracking_id = '/'.join(['hdl:21.14100',uuidVal])
        trg.further_info_url = "https://zenodo.org/record/3957826#.YLgLaahKhPY"
        if not infolder.__contains__("GHGs"):
            trg.source = "Imperial College 4.8.1: Gridded emissions data produced by modifying " + old_long_scen_name + " scenario with the effects of lockdown."
            trg.data_usage_tips = "Note that these are monthly average emissions. Note also that emissions are provided in uneven year intervals."
        else:
            trg.source = "Imperial College 4.8.1: GHG data produced by modifying " + old_long_scen_name + " scenario with the effects of lockdown."
            trg.data_usage_tips = "Note that these are monthly average concentrations."
        db.close()
        trg.close()


Skip file cut_BC-em-anthro_input4MIPs_emissions_ScenarioMIP_IAMC-MESSAGE-GLOBIOM-ssp245-1-1_gn_201501-210012.nc4year_v4.8.1.nc_baseline.nc
Skip file cut_BC-em-anthro_input4MIPs_emissions_ScenarioMIP_IAMC-MESSAGE-GLOBIOM-ssp245-1-1_gn_201501-210012.ncv4.8.1.nc_baseline.nc
TwoYearBlip
BC-em-anthro_input4MIPs_emissions_DAMIP_ImperialCollege-ssp245-covid-v4-8-1_gn_201501-205012.nc


C:\Users\rlamboll\AppData\Local\Continuum\anaconda3\envs\netcdf-5\lib\site-packages\ipykernel_launcher.py:70: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
C:\Users\rlamboll\AppData\Local\Continuum\anaconda3\envs\netcdf-5\lib\site-packages\ipykernel_launcher.py:81: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
C:\Users\rlamboll\AppData\Local\Continuum\anaconda3\envs\netcdf-5\lib\site-packages\ipykernel_launcher.py:88: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
C:\Users\rlamboll\AppData\Local\Continuum\anaconda3\envs\netcdf-5\lib\site-packages\ipykernel_launcher.py:99: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.


Skip file cut_CO-em-anthro_input4MIPs_emissions_ScenarioMIP_IAMC-MESSAGE-GLOBIOM-ssp245-1-1_gn_201501-210012.nc4year_v4.8.1.nc_baseline.nc
Skip file cut_CO-em-anthro_input4MIPs_emissions_ScenarioMIP_IAMC-MESSAGE-GLOBIOM-ssp245-1-1_gn_201501-210012.ncv4.8.1.nc_baseline.nc
TwoYearBlip
CO-em-anthro_input4MIPs_emissions_DAMIP_ImperialCollege-ssp245-covid-v4-8-1_gn_201501-205012.nc
Skip file cut_NH3-em-anthro_input4MIPs_emissions_ScenarioMIP_IAMC-MESSAGE-GLOBIOM-ssp245-1-1_gn_201501-210012.nc4year_v4.8.1.nc_baseline.nc
Skip file cut_NH3-em-anthro_input4MIPs_emissions_ScenarioMIP_IAMC-MESSAGE-GLOBIOM-ssp245-1-1_gn_201501-210012.ncv4.8.1.nc_baseline.nc
TwoYearBlip
NH3-em-anthro_input4MIPs_emissions_DAMIP_ImperialCollege-ssp245-covid-v4-8-1_gn_201501-205012.nc
Skip file cut_NMVOC-em-anthro_input4MIPs_emissions_ScenarioMIP_IAMC-MESSAGE-GLOBIOM-ssp245-1-1_gn_201501-210012.nc4year_v4.8.1.nc_baseline.nc
Skip file cut_NMVOC-em-anthro_input4MIPs_emissions_ScenarioMIP_IAMC-MESSAGE-GLOBIOM-ssp245-1-1_

In [4]:
# Test that we have successfully created a variable with named attributes
test = nc.Dataset(outfolder + "mole-fraction-of-carbon-dioxide-in-air_input4MIPs_GHGConcentrations_DAMIP_ImperialCollege-ssp245-covid-v4-8-1_gr1-GMNHSH_201501-250012.nc")

In [6]:
test

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    title: UoM-MESSAGE-GLOBIOM-ssp245-1-2-1: MESSAGE-GLOBIOM-ssp245 GHG concentrations: global and hemispheric means of CO2 prepared for input4MIPs
    dataset_category: GHGConcentrations
    activity_id: input4MIPs
    Conventions: CF-1.6
    creation_date: 2021-06-03T00:09:45Z
    frequency: mon
    realm: atmos
    tracking_id: hdl:21.14100/6e8b2bd4-ecef-4b95-aae9-a362e4b354f9
    product: assimilated observations; forward projections produced with MAGICC7.0; see www.magicc.org
    mip_era: CMIP6
    table_id: input4MIPs
    grid: global and hemispheric means - area-averages from the original latitudinal 15-degree bands
    grid_label: gr1-GMNHSH
    nominal_resolution: 10000 km
    variable_id: mole_fraction_of_carbon_dioxide_in_air
    comment: Note: Zonal means for 15-degree lat bands or 0.5-degree lat bands available in _gr0p5x360_ or _gr15x360_ files. Here: _GM: global mean; _NH: northern hemi

In [7]:
i=0
version = versions_list[i]
files = os.listdir(infolder_list[i])
files = [file for file in files if (file.__contains__(version))]
db = nc.Dataset(os.path.join(infolder_list[i], files[0]))
db.variables["lon"][:]

masked_array(data=[-179.75, -179.25, -178.75, -178.25, -177.75, -177.25,
                   -176.75, -176.25, -175.75, -175.25, -174.75, -174.25,
                   -173.75, -173.25, -172.75, -172.25, -171.75, -171.25,
                   -170.75, -170.25, -169.75, -169.25, -168.75, -168.25,
                   -167.75, -167.25, -166.75, -166.25, -165.75, -165.25,
                   -164.75, -164.25, -163.75, -163.25, -162.75, -162.25,
                   -161.75, -161.25, -160.75, -160.25, -159.75, -159.25,
                   -158.75, -158.25, -157.75, -157.25, -156.75, -156.25,
                   -155.75, -155.25, -154.75, -154.25, -153.75, -153.25,
                   -152.75, -152.25, -151.75, -151.25, -150.75, -150.25,
                   -149.75, -149.25, -148.75, -148.25, -147.75, -147.25,
                   -146.75, -146.25, -145.75, -145.25, -144.75, -144.25,
                   -143.75, -143.25, -142.75, -142.25, -141.75, -141.25,
                   -140.75, -140.25, -139.75, -139.